In [1]:
import os
import nltk
import string
import numpy as np

In [2]:
directory = "Humor,Hist,Media,Food"

In [3]:
docs = os.listdir(directory)

In [4]:
docs

['1st_aid.txt',
 'a-team',
 'abbott.txt',
 'aboutada.txt',
 'acetab1.txt',
 'aclamt.txt',
 'acne1.txt',
 'acronym.lis',
 'acronym.txt',
 'acronyms.txt',
 'adameve.hum',
 'adcopy.hum',
 'addrmeri.txt',
 'admin.txt',
 'adrian_e.faq',
 'ads.txt',
 'adt_miam.txt',
 'advrtize.txt',
 'aeonint.txt',
 'age.txt',
 'aggie.txt',
 'aids.txt',
 'airlines',
 'alabama.txt',
 'alcatax.txt',
 'alcohol.hum',
 'alflog.txt',
 'allfam.epi',
 'allusion',
 'all_grai',
 'amazing.epi',
 'ambrose.bie',
 'amchap2.txt',
 'analogy.hum',
 'aniherb.txt',
 'anime.cli',
 'anime.lif',
 'anim_lif.txt',
 'annoy.fascist',
 'anorexia.txt',
 'answers',
 'anthropo.stu',
 'antibiot.txt',
 'antimead.bev',
 'aphrodis.txt',
 'appbred.brd',
 'appetiz.rcp',
 'applepie.des',
 'apsaucke.des',
 'apsnet.txt',
 'arab.dic',
 'arcadian.txt',
 'argotdic.txt',
 'arnold.txt',
 'art-fart.hum',
 'arthriti.txt',
 'ateam.epi',
 'atherosc.txt',
 'atombomb.hum',
 'att.txt',
 'aussie.lng',
 'avengers.lis',
 'awespinh.sal',
 'ayurved.txt',
 'a_fish

# Preprocessing the data

In [5]:
def preprocessing(data):
    #Convert text to lowercase
    lowercase_data = data.lower()
    #print("=============Lowercase data==============")
    #print(lowercase_data)
    
    #  Removing Punctuation Marks and Word Tokenization
#     tokenizer = nltk.RegexpTokenizer(r"\w+")
#     token_list = tokenizer.tokenize(lowercase_data)
    token_list = nltk.word_tokenize(lowercase_data)
 
    #print("=============Tokens==============")
    #print(token_list)

    
    #removing stopwords
    stopwords = set(nltk.corpus.stopwords.words('english'))
    tokens = []
    for w in token_list:
        if w not in stopwords:
            tokens.append(w)
    
    #print("=============Tokens with stopwords removed==============")
    #print(tokens)
    
    # Remove punctuations.
    table = str.maketrans('', '', '\t')
    token_list = [word.translate(table) for word in tokens]
    #token_list = [word for word in tokens if word.isalnum()]
    punctuations = (string.punctuation).replace("'", "")
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in token_list]
    token_list = [str for str in stripped_words if str]

    
    
    #print("=============Tokens with punctuation removed==============")
    #print(token_list)
    
    #Removing blank tokens
    while("" in token_list) :
        token_list.remove("")
    
 
    return token_list

# Finding Jaccard's Coefficient

In [6]:
def findJaccardCoefficient(query, doc):
    query_tokens = preprocessing(query)
    doc_tokens = preprocessing(doc)
    jaccard_coeff = len(set(query)&set(doc))/len(set(query)|set(doc))
    return jaccard_coeff

### Fetching top five documents as per the Jaccard Coefficient

In [7]:
query = input("Enter the query : ")
docsDict = {}
jaccardCoeffOfDocs = []
i=0
# Find Jaccard Coefficient for each document and put it in in a list
for file in docs:
    docsDict[i]=file
    filepath = directory+"/"+file
    doc = open(filepath, 'r', encoding ="ascii", errors ="surrogateescape")
    data = doc.read()
    jaccardCoeff = findJaccardCoefficient(query,data)
    jaccardCoeffOfDocs.append(jaccardCoeff)
#     print(file,"\t\t", jaccardCoeff*100)
    i+=1
jaccardCoeffOfDocs = np.array(jaccardCoeffOfDocs)
#Fetching the top 5 documents according to jaccard coefficient
topFiveDocs = jaccardCoeffOfDocs.argsort()[-5:][::-1]
for i in range(len(topFiveDocs)):
    print(docsDict[topFiveDocs[i]])

Enter the query : good day
boston.geog
johann
justify
bad.jok
humpty.dumpty


# Finding TF-IDF

In [27]:
import pandas as pd
import operator
import glob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import copy

In [9]:
docCount = 0
# frequency = pd.DataFrame()
fileIdName = []
fileTokens= []

for file in docs:
    filepath = directory+"/"+file
    fileIdName.append([file])
    file = open(filepath, 'r',encoding ="ascii", errors= 'surrogateescape')
    data = " ".join(file.read().split()) 
    file.close()
    preprocessed_data = preprocessing(data)
    fileTokens.append(preprocessed_data)
fileIdName = pd.DataFrame(fileIdName)

In [28]:
#1-Raw Count-f(t,d)-----frequency of each term in doc

RawFreqCount = []
for itm in fileTokens:
    raw_tf_dict ={}
    for w in itm:
        if raw_tf_dict.get(w) == None:  
            raw_tf_dict[w] = 1
        else:
            raw_tf_dict[w] = raw_tf_dict.get(w)+1
    RawFreqCount.append(raw_tf_dict) 

In [15]:
#2-f(t,d)/∑f(t`,d)---frequncy of term divide by all no. of words in doc
Term_Frequency = []     
for itm in RawFreqCount:
    tf_dict = {}
    sum_tf = sum(itm.values())     
    for w in itm.keys(): 
        tf_dict[w] = (itm.get(w)/sum_tf) 
    Term_Frequency.append(tf_dict)

In [32]:
#3- Log Normalisation = log(1 +f(t,d)))
Log_Normalisation = []       
for itm in RawFreqCount:
    log_tf_dict = {}        
    allkeys = itm.keys()
    for w in allkeys:  
        log_tf_dict[w] =  np.log(1+itm.get(w))     
    Log_Normalisation.append(log_tf_dict)

In [34]:
#4---Double Normalisation Frequency = 0.5 + ((0.5*tf)/max(tf))
Double_Normalisation = []  
for itm in RawFreqCount:
    dn_tf_dict = {}             
    allkeys = itm.keys()
    max_tf = max(itm.values())
    for w in allkeys:  
        dn_tf_dict[w] = 0.5 + (0.5*(itm.get(w)/max_tf)) 
    Double_Normalisation.append(dn_tf_dict)

In [36]:
#5------Binary 0,1
Binary_Term_Frequency = []  
for itm in RawFreqCount:
    bin_tf_dict = {}      
    allkeys = itm.keys()
    for w in allkeys:
        freq = itm.get(w)
        if freq >= 1 :          
            bool_freq = 1 
        else:
            bool_freq = 0  
        bin_tf_dict[w] = bool_freq
    Binary_Term_Frequency.append(bin_tf_dict)

In [40]:
#Document Frequency --no. of docs in which term t present
docFreq = {}   
for itm in fileTokens:
    termsList = list(set(itm))
    for wrd in termsList:
        if docFreq.get(wrd) == None:
            docFreq[wrd] = 1  
        else: 
            docFreq[wrd] = docFreq.get(wrd)+1

In [42]:
print(len(docFreq))
N = len(docs)
print('The Total Number of docs are : ',N)

84064
The Total Number of docs are :  1133


In [44]:
idf = {}
keys_df = docFreq.keys()
for item in keys_df:
    idf[item] = np.log(N/docFreq[item] +1)

In [51]:
#Function block for Calculating tf-idf value---tf*idf as tf=idf will be calculated based on all the 5 variants
def get_tfidf(list_tf):
    tfidf_list_resultant = []  
    for itm in list_tf:
        tfidf_dict = {}
        allkeys = itm.keys()
        for w in allkeys:  
            tfidf = itm[w]  * idf[w]       
            tfidf_dict[w] = tfidf
        tfidf_list_resultant.append(tfidf_dict)
    return tfidf_list_resultant

In [52]:
#calculating tfidf for all 5 variants
binary_tfidf = get_tfidf(Binary_Term_Frequency)
raw_count_tfidf = get_tfidf(RawFreqCount)
term_frequency_tfidf = get_tfidf(Term_Frequency)
log_normalisation_tfidf = get_tfidf(Log_Normalisation)
double_normalisation_tfidf = get_tfidf(Double_Normalisation)

In [53]:
len(raw_count_tfidf)
all_terms = list(docFreq.keys())
len(all_terms)

84064

In [64]:
#tf-idf score using log
log_matrix_tfidf = []  
for itm in log_normalisation_tfidf:
    log_doc = []
    for w in all_terms:
        if w in itm :
            log_doc.append(itm[w])  
        else:
            log_doc.append(0.0) 
    log_matrix_tfidf += [log_doc]   

In [65]:
#tf-idf score using binary
binary_matrix_tfidf = []  
for itm in binary_tfidf:
    bin_doc = []
    for w in all_terms:
        if w in itm :
            bin_doc.append(itm[w]) 
        else:
            bin_doc.append(0.0)   
    binary_matrix_tfidf += [bin_doc]   

In [66]:
#tf-idf score using double
double_matrix_tfidf = []  
for itm in double_normalisation_tfidf:
    db_doc = []
    for w in all_terms:
        if w in itm :
            temp = itm[w]
            db_doc.append(temp) 
        else:
            db_doc.append(0.0)  
    double_matrix_tfidf += [db_doc]   

In [67]:
##tf-idf score using term frequency
term_matrix_tfidf = []   
for itm in term_frequency_tfidf:
    term_doc = []
    for w in all_terms:
        if w in itm :
            term_doc.append(itm[w])  
        else:
            term_doc.append(0.0)
    term_matrix_tfidf += [term_doc]   

In [70]:
#tf-idf score using raw count
raw_matrix_tfidf = [] 
for itm in raw_count_tfidf:
    inside_doc = []
    for word in all_terms:
        if word in itm :
            inside_doc.append(itm[word]) 
        else:
            inside_doc.append(0.0) 
    raw_matrix_tfidf += [inside_doc]        

In [85]:
#fetching rank document based on tfidf
def rank_document_tfidf(variant_list_tfidf, queries):
    queryVector = [0.0] * len(all_terms)
    t = set(queries)
    res = [i for i, val in enumerate(all_terms) if val in t]

    rank_doc = {}
    for i in range(0,N):
        rank_doc[i] = 0.0
    for query_word in res:
        count = -1
        for itm in variant_list_tfidf:
            count += 1
            rank_doc[count] += itm[query_word]  
    
    for i in range(len(all_terms)):
        if i in res:
            queryVector[i]=1.0
        else:
            queryVector[i]=0.0


    return dict(sorted(rank_doc.items(),key=operator.itemgetter(1),reverse=True)),queryVector

In [86]:
def top_five_doc_tfidf(tfidf_list, queries, k, var):
    #print(list_tfidf)
    list_tf ,query_vector= rank_document_tfidf(tfidf_list,queries)
    count = -1
    flag = 0
    list_final = []
    for item in list_tf.keys():
        count += 1
        if count == k:
            print('Top ',k,' Documents based on ',var,' tf-idf are',list_final)
            for i in list_final:
                print(fileIdName.iloc[i][0])
            flag = 1
            break
        list_final.append(item)
    if flag == 0:
        print('Top ',k,' Documents based on ',var,' tf-idf are:',list_final)    
        for i in list_final:
            print(fileIdName.iloc[i][0])
    return query_vector         

In [87]:
#query token is calculated
def query_tokens(query):
    return word_tokenize(str(preprocessing(query)))

In [88]:
query = input('Enter the String   : ')
query_afterPreprocess =query_tokens(query) 
query_vector_raw=top_five_doc_tfidf(raw_matrix_tfidf,query_afterPreprocess,5,'Raw_Count')
query_vector_log=top_five_doc_tfidf(log_matrix_tfidf,query_afterPreprocess,5,'Logarithmic')   
query_vector_term=top_five_doc_tfidf(term_matrix_tfidf,query_afterPreprocess,5,'termfrequency')
query_vector_binary=top_five_doc_tfidf(binary_matrix_tfidf,query_afterPreprocess,5,'Binary')
query_vector_double=top_five_doc_tfidf(double_matrix_tfidf,query_afterPreprocess,5,'Double') 

Enter the String   : good day
Top  5  Documents based on  Raw_Count  tf-idf are [519, 520, 977, 516, 663]
insult.lst
insults1.txt
strine.txt
insanity.hum
mlverb.hum
Top  5  Documents based on  Logarithmic  tf-idf are [519, 520, 222, 977, 516]
insult.lst
insults1.txt
clancy.txt
strine.txt
insanity.hum
Top  5  Documents based on  termfrequency  tf-idf are [963, 516, 1102, 520, 239]
spelin_r.ifo
insanity.hum
wood
insults1.txt
cold.fus
Top  5  Documents based on  Binary  tf-idf are [222, 1, 2, 6, 7]
clancy.txt
a-team
abbott.txt
acne1.txt
acronym.lis
Top  5  Documents based on  Double  tf-idf are [222, 84, 181, 239, 242]
clancy.txt
basehead.txt
calif.hum
cold.fus
college.hum
